In [1]:
from utils.agents.agent_graph import MyAgent
import os

/Users/debasmitroy/Desktop/programming/agent-starter-1/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# For Gemini
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/debasmitroy/Desktop/programming/temp_deployment/key.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "openserve-0"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1" 

## Agent Compile

In [3]:
MY_AGENT = MyAgent(thread_id=1)

In [4]:
graph = MY_AGENT.get_graph()
print(graph.draw_ascii())

                              +-----------+                         
                              | __start__ |                         
                              +-----------+                         
                                    *                               
                                    *                               
                                    *                               
                                +-------+                           
                              ..| start |..                         
                          ....  +-------+  ....                     
                      ....           .         .....                
                  ....                .             ....            
               ...                    .                 ....        
+-------------------+         +----------------+            ...     
| write_python_code |         | write_sql_code |              .     
+-------------------+         +---

## Running Agent

### Example 1

In [5]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a python code to find the sum of two numbers.",
        "write_python_code": "",
        "write_sql_code": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 11:17:30,960 - INFO ==> Starting the agent-assist
2025-03-20 11:17:30,961 - INFO ==> Checking user query
2025-03-20 11:17:31,208 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1
2025-03-20 11:17:33,385 - INFO ==> Writing python code
2025-03-20 11:17:33,657 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1
2025-03-20 11:17:36,027 - INFO ==> Running the python code
2025-03-20 11:17:36,029 - INFO ==> Code executed successfully: 8

2025-03-20 11:17:36,031 - INFO ==> Summarizing the code
2025-03-20 11:17:36,331 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1
2025-03-20 11:17:37,461 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': 'Write a python code to find the sum of two numbers.',
  'write_python_code': 'def sum_two_numbers(a, b):\n  return a + b\n\n# Example usage:\nnum1 = 5\nnum2 = 3\nresult = sum_two_numbers(num1, num2)\nprint(result)',
  'write_sql_code': '',
  'summary': 'The user asked for a Python code to find the sum of two numbers and provided the code as: def sum_two_numbers(a, b):\n  return a + b\nnum1 = 5\nnum2 = 3\nresult = sum_two_numbers(num1, num2)\nprint(result)',
  'python_code_output': '8\n'}}

In [6]:
print("*** The code is: ", result_state['results']['write_python_code'])
print("*** The output is: ", result_state['results']['python_code_output'])
print("*** The summary is: ", result_state['results']['summary'])

*** The code is:  def sum_two_numbers(a, b):
  return a + b

# Example usage:
num1 = 5
num2 = 3
result = sum_two_numbers(num1, num2)
print(result)
*** The output is:  8

*** The summary is:  The user asked for a Python code to find the sum of two numbers and provided the code as: def sum_two_numbers(a, b):
  return a + b
num1 = 5
num2 = 3
result = sum_two_numbers(num1, num2)
print(result)


### Example 2

In [7]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a sql code to fetch the all data from the table 'employee'.",
        "write_python_code": "",
        "write_sql_code": "",
        "python_code_runner": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 11:17:37,482 - INFO ==> Starting the agent-assist
2025-03-20 11:17:37,482 - INFO ==> Checking user query
2025-03-20 11:17:39,644 - INFO ==> Writing sql code
2025-03-20 11:17:39,863 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1
2025-03-20 11:17:40,642 - INFO ==> Running the sql code
2025-03-20 11:17:40,643 - WARNING ==> SQL code runner is not implemented yet. Need to register the SQL engine first.
2025-03-20 11:17:40,646 - INFO ==> Summarizing the code
2025-03-20 11:17:42,991 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': "Write a sql code to fetch the all data from the table 'employee'.",
  'write_python_code': '',
  'write_sql_code': 'SELECT * FROM employee;',
  'python_code_runner': '',
  'summary': "The user asked for SQL code to fetch all data from the 'employee' table, and the provided code is 'SELECT * FROM employee;'.",
  'sql_code_output': 'SQL code runner is not implemented yet. Need to register the SQL engine first.'}}

### Example 3

In [8]:
initial_state = {
    "state": "start",
    "results": {
        "user": "Write a ruby code",
        "write_python_code": "",
        "write_sql_code": "",
        "summary": ""
    }
}

result_state = MY_AGENT.continue_flow(initial_state)
result_state

2025-03-20 11:17:43,012 - INFO ==> Starting the agent-assist
2025-03-20 11:17:43,013 - INFO ==> Checking user query
2025-03-20 11:17:44,828 - WARNING ==> Rate limit hit. Retrying in 30 seconds... (Attempt 1/5)
2025-03-20 11:18:16,679 - WARNING ==> Rate limit hit. Retrying in 30 seconds... (Attempt 2/5)
2025-03-20 11:18:48,935 - ERROR ==> Error in the user query
2025-03-20 11:18:48,936 - INFO ==> Ending the agent-assist


{'state': 'end',
 'results': {'user': 'Write a ruby code',
  'write_python_code': '',
  'write_sql_code': '',
  'summary': ''}}